# 🛒 Online Retail Recommendation System
This project builds a product recommendation system using online retail transaction data and machine learning techniques (item-based collaborative filtering).

In [ ]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Step 2: Load Dataset (CSV File)
df = pd.read_csv("OnlineRetail.csv")  # Make sure file is in same folder
print("✅ Data Loaded Successfully")

In [ ]:
# Step 3: Basic Data Exploration
print(df.shape)
print(df.columns)
print(df.head())

# Check null values
print("\nNull values in each column:")
print(df.isnull().sum())

In [ ]:
# Step 4: Data Cleaning

# Remove missing Customer IDs
df.dropna(subset=['CustomerID'], inplace=True)

# Remove negative or 0 quantity transactions
df = df[df['Quantity'] > 0]

# Remove duplicates
df.drop_duplicates(inplace=True)

# Remove rows with missing description
df.dropna(subset=['Description'], inplace=True)

# Remove unwanted spaces in product names
df['Description'] = df['Description'].str.strip()

print("✅ Data Cleaned")

In [ ]:
# Step 5: Optional - Filter Frequent Products

popular_products = df['Description'].value_counts()
df = df[df['Description'].isin(popular_products[popular_products > 50].index)]

print("✅ Rare products removed (optional step)")

In [ ]:
# Step 6: Create Customer-Product Matrix

basket = df.pivot_table(index='CustomerID', 
                        columns='Description', 
                        values='Quantity', 
                        aggfunc='sum').fillna(0)

print("✅ Customer-Product Matrix Created")

In [ ]:
# Step 7: Convert to Sparse Matrix & Compute Similarity

sparse_matrix = csr_matrix(basket)

item_similarity = cosine_similarity(sparse_matrix.T)

item_similarity_df = pd.DataFrame(item_similarity, 
                                  index=basket.columns, 
                                  columns=basket.columns)

print("✅ Item Similarity Matrix Ready")

In [ ]:
# Step 8: Recommend Function

def recommend_items(product_name, num_recommendations=5):
    product_name = product_name.strip()
    if product_name not in item_similarity_df.columns:
        return "❌ Product not found in dataset."

    print(f"\n📢 Recommendations for: {product_name}\n")
    similar_items = item_similarity_df[product_name].sort_values(ascending=False)[1:num_recommendations+1]

    for i, (item, score) in enumerate(similar_items.items(), 1):
        print(f"{i}. {item} (Similarity Score: {score:.2f})")

In [ ]:
# Step 9: Test the Recommender

# Replace with a real product name from the dataset
test_product = "WHITE HANGING HEART T-LIGHT HOLDER"
recommend_items(test_product, num_recommendations=5)